# RSA signatures

Sigurjón Ágústsson

In [1]:
import random
import time
import numpy as np

In [2]:
def load_small_primes():
    pfile =  open("1000primes.txt", "r")
    small_primes = [int(num) for num in pfile.read().split(',')]
    return small_primes

small_odd_primes = load_small_primes()
print("Small odd primes loaded : {}...{}".format(small_odd_primes[:5], small_odd_primes[-3:]))

Small odd primes loaded : [3, 5, 7, 11, 13]...[7901, 7907, 7919]


We start by introducing some helper functions. Most important of which is the modular exponentiation function which uses Montgomery modular multiplication to very quickly compute modular exponentiation.

In [3]:
def even(n):
    """Checks the parity of n"""
    return (n&1) != 1

def pow_n_mod_n(a,n):
    """Naive and very inefficient way of doing modular exponentiation"""
    return ((a**(n-1)) % n)

def modular_exponentiation(a,n):
    """Python's fast way of doing modular exponentiation"""
    return pow(a,(n-1),n)

def divisible_by_small_prime(n, resolution=1000):
    """A function that checks if a number n has one of the 
        1000 first primes as a factor (excluding 2 because 
        it is checked for already)"""
    global small_odd_primes # The array of primes
    for small_prime in small_odd_primes[:resolution]:
        if n == small_prime:
            # In this case n is a small prime number, and we return
            # False to indicate it is not divisible by any of
            # the smaller primes
            return False
        if (n % small_prime) == 0:
            # print("diviseble by small prime")
            return True
    return False

def seconds_to_prefix(sec):
    """Just a function that inputs number of seconds (float) and 
        outputs a string representation with a unit (s, ms or min) """
    if sec < 1:
        ms = sec * 1000
        return "{} ms".format(round(ms,4))
    elif sec < 60:
        return "{} s".format(round(sec,4))
    else:
        min = int(sec) // 60
        rem = seconds_to_prefix(sec-60*min)
        return "{} min and {}".format(min,rem)

Let's demonstrate with a timing example.

In [4]:
bits = 20
# Let n be a random 20 bit integer
n = random.randint((1<<(bits-1)),((1<<bits)-1))
a = random.randint(0xFFFF,n-1) # a very large number but still less than n


ex1_start = time.perf_counter()
print(pow_n_mod_n(a,n))
ex1_end = time.perf_counter()
ex2_start = time.perf_counter()
print(modular_exponentiation(a,n))
ex2_end = time.perf_counter()
t1,t2 = (ex1_end-ex1_start), (ex2_end-ex2_start)
strt1,strt2 = seconds_to_prefix(t1), seconds_to_prefix(t2)
diff = round(t1/t2,2)
print("Using the slow algorithm, it took: {} \
        \nUsing the fast algorithm, it took: {}".format(strt1,strt2))
print("The Montgomery algorithm makes modular exponentiation {} times faster in this case!".format(diff))

83344
83344
Using the slow algorithm, it took: 1.6449 s         
Using the fast algorithm, it took: 0.0423 ms
The Montgomery algorithm makes modular exponentiation 38886.88 times faster in this case!


## Prime number generation

Now we are ready to do generate prime numbers. 

In [5]:
def probable_prime(size):
    assert size > 0
    s = size-1
    smallest = 1<<s
    largest = (1<<(s+1))-1
    while True:
        # First pick a random number n of size 'size' bits
        n = random.randint(smallest,largest)
        if even(n) or divisible_by_small_prime(n):
            continue
        a = random.randint(2,n-1)

        mod_computation = modular_exponentiation(a,n)

        if mod_computation == 1:
            # By Fermat's little thm, if the result is not 1
            # then n is not prime, otherwise, it is likely to 
            # be prime
            return n

Let's use this to generate some "small" probably-prime numbers, or pseudoprimes.

In [6]:
for i in range(4):
    bits = 8*(i+1)
    print("{} bits: \t".format(bits), probable_prime(bits))

8 bits: 	 241
16 bits: 	 55529
24 bits: 	 8897221
32 bits: 	 2699350453


In RSA encryption we want much larger primes. Typically 1024 or 3072 bits. That is no obstacle for us, let's generate such pseudoprimes:

In [7]:
p1024 = probable_prime(1024)
p3072 = probable_prime(3072)

In [8]:
p1024

124722207715798843085314418188174158449674016073167808559486344560870871416305403611896896302537506865104753414437387078722960057176336220492800624133246357933834268693150785848549239637891063220475000201829481360915812288886503918885610733347578137034563799007994077297748361290036179303969911907362561827053

In [9]:
p3072

4852670007218978024068530440384602121336294282306837767698195092915570978909080639858255217683445293958453732230658816203108738918375458390226917098623096968785784136882216366869838497921447232217780081156076312982166317042725799872624217937286944394718757919520162281138089137162427745210408196685859710370602914071472756735537942621530081757225213174020416757502112548790138972181691584451502195097938801313482534482328767197405952795332867422785149505561044668317330468449415182979929036180595294609872069809028005299298734409742629386038908368734005272952472018598513546116905772154904208065428673287161154182772022891996011445423309870830400781171486575268305511571720833534875972961660803304710776256946822526700962142571991630637782424846601056759081283638508604705693202451586225402060949315667589682143584545055279982721800973138046436284713029715459108446069301884182952052910100257843972580215290708143327107080627

## RSA signatures

In [10]:
def find_ed(N,T):
    e = 3
    d = 4
    while (e*d)%T != 1:
        if d>2000:
            raise ValueError("d is too large this is not working")
        d += 1
    return (e,d)

In [26]:
def generate_keys(size=1024):
    """Function that returns a tuple of public and private keys of given size (in bits)"""
    p,q = probable_prime(size),probable_prime(size)
    N = p*q
    T = N - p - q + 1 # (p-1)(q-1)
    # e is less than T. And it is coprime to T and N
    # e*d mod T has to be 1
    # e is typically 3 - Ben
    e,d = find_ed(N,T)
    return ((N,e),(N,d))

def dummy_keys():
    return ((91, 5), (91, 29))

def encrypt(val, public_key):
    """public key is a tuple containing N,e """
    N,e = public_key
    return pow(val,e,N)

def decrypt(cipher, private_key):
    """Private key is a tuple containing N,d """
    N,d = private_key
    return pow(cipher,d,N)

In [27]:
# public_key, secret_key = generate_keys()
public_key, secret_key = dummy_keys()

In [28]:
public_key

(91, 5)

In [29]:
secret_key

(91, 29)

In [30]:
text = "hello world!"
l = ["a","b","c"]
ord('\x04')

4

In [44]:
def str2ascii_list(string):
    return [ord(c) for c in string]

def ascii_list2str(alist):
    s = ""
    for c in alist:
        s+=chr(c)
    return s

def encrypt_string(string, public_key):
    ascii_list = [ord(i) for i in string]
    enc_list = [encrypt(j,public_key) for j in ascii_list]
    print(enc_list)
    enc_string  = ""
    for enc in enc_list:
        enc_string += chr(enc)
    return enc_list

def decrypt_string(enc_list,private_key):
    ascii_list = [decrypt(j, private_key) for j in enc_list]
    return ascii_list

In [46]:
text_asc = str2ascii_list(text)
print("original:")
print(text_asc)
print("encrypted:")
enc = [encrypt(c, public_key) for c in text_asc]
print(enc)
print("decrypted_list")
dec_list = [decrypt(c, secret_key) for c in enc]
print(dec_list)
print("result")
res = ascii_list2str(dec)
print(res)

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 33]
hello world!


In [41]:
char = ord('A')
enc_char = encrypt(char, public_key)
print(enc_char)
dec_char = decrypt(enc_char,secret_key)
print(dec_char)
print(chr(dec_char))

39
65
A


In [42]:
encrypted = encrypt_string(text,public_key)
print(encrypted)
decrypted = decrypt_string(encrypted,secret_key)
print(decrypted)

[13, 82, 75, 75, 76, 2, 84, 76, 4, 75, 81, 24]
[13, 82, 75, 75, 76, 2, 84, 76, 4, 75, 81, 24]
[13, 10, 17, 17, 20, 32, 28, 20, 23, 17, 9, 33]
